There are a few issues with loading the data, some improperly formatted rows, if choosing to ignore the bad rows it contains 9373 rows in total from a CSV that had 10688 rows, rather large amount due to mismatch between number of items on a line and the number of columns, due to line breaks in the descriptions only 17 of the lines were badly formatted.

A cause for concern there is tweet data for 9662 users but the user table only contains 9380 users.

In [3]:
from gender_detector.gender_detector import GenderDetector as gd
import numpy as np
import pandas as pd
import re
import separateGUsers as sgu

#twitter_Users = pd.read_csv("usersTwitter.csv", sep=",", encoding="utf-8", index_col=False, usecols=range(20))
#twitter_Users.head()

# Check For Duplicate Users and Tweets

In [38]:
twitter_Users.duplicated().value_counts()

False    8610
True      770
dtype: int64

770 duplicate rows in the user frame, will need to be dropped

In [5]:
tweets = pd.read_csv("usertestTweets/jooseboks.csv", sep=",", encoding="utf-8", index_col=False, usecols=range(24))
tweets.duplicated().value_counts()

False    170
dtype: int64

Take list of google users, compare to twitter users to ensure one to one correlation and removing pages and keeping people. Select columns that are of interest and relevant to the model and write to file

__Don't run cell below more than once__

In [15]:
users = sgu.read_dict_pairs()

twitter_users = pd.read_csv("usersTwitter.csv", sep=",", encoding="utf-8", index_col=False, usecols=range(20))
twFix = twitter_users[twitter_users.screen_name.isin(users.values())].drop_duplicates()
twFix = twFix[["id_str", "name", "screen_name", "location", "description", "url", "followers_count", "friends_count", "listed_count", "statuses_count", "lang"]]
twFix.set_index("id_str").to_csv("usersTrueTwitter.csv", index_label="id_str", encoding="utf-8")


Read in filtered twitter users

In [23]:
users = pd.read_csv("usersTrueTwitter.csv", sep=",", index_col="id_str", encoding="utf-8", engine='python')

Play around with the gender guesser thing. Picked US because it is likely to include spanish names as well

In [17]:
test = gd("us")
test.guess("Emilia Miller")
name = users.iloc[0,0]
fname = name.split()[0]
test.guess(fname) #Justin
test.guess("the")

'male'

Try out regular expression match first token of only characters in case some punctuation or numbers are mixed in there

In [18]:
reg = re.compile("[a-z]*", re.IGNORECASE)
match = reg.match("Ber.t J. Gives, Jr.")


Assign genders to the twitter users using the package. Haven't created checks for accuracy but should. The name string is tokenised and the first token is match to the regexp and guessed. Sometimes the thing throws KeyErrors (disallowed chars) and IndexErrors (empty string).

__Takes a while to run so please save yourself the pain and load in the file I've already prepped. If you overwrite re-run this cell__

In [19]:
genderlist = []
gend = gd("us")
reg = re.compile("[a-z]*", re.IGNORECASE)
for name in users.name:
    try:
        if name is None:
            raise KeyError
        name = name.split()[0] #Take the first name prior to whitespace
        name = reg.match(name).group()
        gender = gend.guess(name)
        genderlist.append(gender)
    except (KeyError, IndexError):
        genderlist.append("unknown")
print("Done")

Done


Write the dataframe to a file, cannot be bothered to run all these cells whenever I want to start

In [20]:
users['gender'] = genderlist
users.to_csv("usersTrueTwitter.csv", index_label="id_str", encoding="utf-8")

Read in both dataframes for the user data

In [24]:
usersG = pd.read_csv("usersGoogle.csv", sep=",", index_col="id", quotechar="\"", encoding="utf-8", engine="c")
usersT = pd.read_csv("usersTrueTwitter.csv", sep=",", index_col="id_str", encoding="utf-8", engine='python')

Encode the Gender Column for both dataframes, dictionary initialised beforehand to see one-hot encoding

In [22]:
encDict = {"unknown":0,"none":0,"other":0,"male":1,"female":2}
usersT['gender_enc'] = usersT.gender.apply(lambda x: encDict[str(x).lower()])
usersG['gender_enc'] = usersG.gender.apply(lambda x: encDict[str(x).lower()])